### Scenario 3: Multiple data scientists working on multiple ML models
MLflow setup:

* Tracking server: yes, remote server (EC2).
* Backend store: postgresql database.
* Artifacts store: s3 bucket.

The experiments can be explored by accessing the remote server.

The exampe uses AWS to host a remote server. In order to run the example you'll need an AWS account. Follow the steps described in the file `mlflow_on_aws.md` to create a new AWS account and launch the tracking server.

In [8]:
#reading aws config and loading into env variable
import os
import configparser
conf = configparser.RawConfigParser()
conf.read('/Users/bupadhy/.aws/credentials')
conf.sections()

aws_access_key_id = conf.get('default','aws_access_key_id')
aws_secret_access_key = conf.get('default','aws_secret_access_key')

os.environ['ACCESS_KEY_ID'] = aws_access_key_id
os.environ['ACCESS_SECRET_KEY'] = aws_secret_access_key

In [10]:
import mlflow

TRACKING_SERVER_HOST = "ec2-15-207-14-226.ap-south-1.compute.amazonaws.com" # fill in with the public DNS of the EC2 instance
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")


In [11]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://ec2-15-207-14-226.ap-south-1.compute.amazonaws.com:5000'


In [ ]:
mlflow.list_experiments()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

In [ ]:
mlflow.list_experiments()

In [ ]:
#interacting with model registry
from mlflow.tracking import MlflowClient


client = MlflowClient(f"http://{TRACKING_SERVER_HOST}:5000")

In [ ]:
client.list_registered_models()

In [ ]:
run_id = client.list_run_infos(experiment_id='1')[0].run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)